## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [14]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o3/o3_bincore8.csv", index_col=0)
print(bin8_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_3.reset_index(inplace=True, drop=True)

print('shape -', bin8_3.shape)
print('reset_index 완료')
print('input data shape')
bin8_3.head()

(31784379, 2)
shape - (31784379, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [15]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_3['label'].value_counts())

256
0    31747485
1       36894
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [16]:
idx_bin = bin8_3[bin8_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_3', len(ls_idx_bin))

# loc 로 수정필요
bin8_3_Ngram = bin8_3.loc[ls_idx_bin,:].copy()

1180608
나머지 0
최종 길이 1180608
bin8_3 1180608


## (4) false data 만들기

In [17]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

36894.0
0 9717497
1000 28122910
2000 26846215
3000 617897
4000 8609419
5000 18411742
6000 29873135
7000 3486338
8000 7247722
9000 9145765
10000 22704801
11000 28910976
12000 19445327
13000 9819012
14000 24359788
15000 16386275
16000 17146965
17000 939242
18000 20239301
19000 12919340
20000 12374161
21000 30913546
22000 13196854
23000 2189409
24000 4431500
25000 5242615
26000 22105933
27000 11226745
28000 141851
29000 30877401
30000 21748809
31000 14109102
32000 23916861
33000 20066619
34000 11530113
35000 3072687
36000 8308737
완료
36894


In [18]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_3['label'].value_counts()[1])

36894
36894


## (5) False Data + True Data 합치기

In [19]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_3_Ngram])
final.shape

(2361216, 2)

## (6) one hot encoding

In [20]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_3_onehot_Ngram = pd.concat([final['label'], bc8_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_3_onehot_Ngram.shape)

원핫인코딩완료
(2361216, 257)


In [21]:
# 훈련 데이터, 훈련 라벨
x_bc8_3 = bc8_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_3 = bc8_3_onehot_Ngram['label'].to_numpy()
print(x_bc8_3.shape, x_bc8_3.shape)

x_bc8_3 = x_bc8_3.reshape(-1, right_idx, x_bc8_3.shape[1])
y_bc8_3 = y_bc8_3.reshape(-1, right_idx, 1)

print(x_bc8_3.shape, y_bc8_3.shape)

(2361216, 256) (2361216, 256)
(73788, 32, 256) (73788, 32, 1)


In [22]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_3.shape[0])

x_bc8_3 = x_bc8_3[p]
y_bc8_3 = y_bc8_3[p]

print(x_bc8_3.shape, y_bc8_3.shape)

(73788, 32, 256) (73788, 32, 1)


## (7) 모델

In [23]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [24]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_3, y_bc8_3):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_3[train],
               y_bc8_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 30s 453us/sample - loss: 0.0149 - accuracy: 0.9961
Epoch 2/10
66409/66409 [==============================] - 21s 319us/sample - loss: 0.0024 - accuracy: 0.9994
Epoch 3/10
66409/66409 [==============================] - 20s 295us/sample - loss: 0.0017 - accuracy: 0.9995
Epoch

accuracy_score 0.9997077856078059
recall_score 0.9930525401650022
precision_score 0.9919757102580785
f1_score 0.9925138331344255
Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 21s 314us/sample - loss: 6.0710e-04 - accuracy: 0.9998
Epoch 2/10
66409/66409 [==============================] - 16s 247us/sample - loss: 4.6979e

accuracy_score 0.999767075484483
recall_score 0.9943107221006564
precision_score 0.9936584299147169
f1_score 0.993984468992672
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 34s 506us/sample - loss: 3.4568e-04 - accuracy: 0.9999
Epoch 2/10
66409/66409 [==============================] - 21s 321us/sample - loss: 2.3589e-0

accuracy_score 0.9998941252202196
recall_score 0.998886910062333
precision_score 0.9955624583980475
f1_score 0.9972219135459496
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 21s 315us/sample - loss: 2.3093e-04 - accuracy: 1.0000
Epoch 2/10
66409/66409 [==============================] - 15s 223us/sample - loss: 2.3724e-

accuracy_score 0.9999534150968966
recall_score 1.0
precision_score 0.9975555555555555
f1_score 0.9987762821225943
Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 23s 339us/sample - loss: 1.7921e-04 - accuracy: 1.0000
Epoch 2/10
66409/66409 [==============================] - 16s 240us/sample - loss: 1.2726e-04 - accuracy:

...

accuracy_score 0.9999830600352351
recall_score 0.9997731911998186
precision_score 0.9993198821128996
f1_score 0.999546485260771
Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 22s 326us/sample - loss: 4.7669e-05 - accuracy: 1.0000
Epoch 2/10
66409/66409 [==============================] - 16s 237us/sample - loss: 2.4143e-

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66409 samples
Epoch 1/10
66409/66409 [==============================] - 34s 512us/sample - loss: 1.9660e-05 - accuracy: 1.0000
Epoch 2/10
66409/66409 [==============================] - 22s 328us/sample - loss: 5.3683e-08 - accuracy: 1.0000
Epoch 3/10
66409/66409 [=============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66410 samples
Epoch 1/10
66410/66410 [==============================] - 21s 317us/sample - loss: 3.4917e-09 - accuracy: 1.0000
Epoch 2/10
66410/66410 [==============================] - 15s 223us/sample - loss: 2.1446e-08 - accuracy: 1.0000
Epoch 3/10
66410/66410 [=============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 66410 samples
Epoch 1/10
66410/66410 [==============================] - 21s 324us/sample - loss: 3.7918e-10 - accuracy: 1.0000
Epoch 2/10
66410/66410 [==============================] - 15s 233us/sample - loss: 3.4535e-10 - accuracy: 1.0000
Epoch 3/10
66410/66410 [=============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9997077856078059, 0.999767075484483, 0.9998941252202196, 0.9999534150968966, 0.9999576500880878, 0.9999830600352351, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9930525401650022, 0.9943107221006564, 0.998886910062333, 1.0, 0.9977533138620535, 0.9997731911998186, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9919757102580785, 0.9936584299147169, 0.9955624583980475, 0.9975555555555555, 1.0, 0.9993198821128996, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9925138331344255, 0.993984468992672, 0.9972219135459496, 0.9987762821225943, 0.9988753936122357, 0.999546485260771, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [25]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999263111532727
10-Fold Cross_validation. Recall : 0.9983776677389864
10-Fold Cross_validation. Precision : 0.9978072036239297
10-Fold Cross_validation. F1-Score : 0.9980918376668647


In [26]:
model1.save('gcc8_bin_core_s32_h48_o3.h5')
print('save 완료')

save 완료
